In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [ ]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from ast import literal_eval
from pandas.core.common import flatten

# Import and Process Data from GeoCov/ARC

In [ ]:
# ARC_PROCESSED_GEOCOV_PATH_ROOT = r"/arc/project/st-tlemieux-1/data/Processed_GeoCov-Data/"
# ARC_PROCESSED_GEOCOV_PATH_ROOT = "C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/Intermediate Cleaned Data/"
# ARC_PROCESSED_GEOCOV_PATH_ROOT = "C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/Aggregating GeoCov Results/Sample Output/"

processed_geocov_files = [f for f in listdir(ARC_PROCESSED_GEOCOV_PATH_ROOT) if isfile(join(ARC_PROCESSED_GEOCOV_PATH_ROOT, f))]





# DELETE THIS!!!!!!!!!!!!!!!!
# processed_geocov_files = processed_geocov_files[:3] 

In [ ]:
geocov_df = pd.concat((pd.read_csv(path) for path in [ARC_PROCESSED_GEOCOV_PATH_ROOT + date for date in processed_geocov_files]))

In [ ]:
# geocov_df

In [ ]:
del geocov_df['Unnamed: 0']

In [ ]:
geocov_df = geocov_df.fillna('').groupby('user_id').agg(list)

In [ ]:
# def literal(x):
#     if not pd.isnull(x):
#         return literal_eval(x)
#     else:
#         return x

In [ ]:
def clean_list(li):
    if li == []:
        
        return np.nan
    
    else:
        
        local_list = list(filter(lambda a: a != '', li))
        
        local_list = np.ravel(local_list).tolist()
        
        local_list = [x.replace('.0', '') for x in local_list]
        
        try:
            local_list = list(flatten([literal_eval(x) for x in local_list]))
        except:
            return []
            print(local_list)
#         local_list = [int(float(x.replace('[','').replace(']',''))) for x in local_list]
        
        return local_list        

In [ ]:
# geocov_df

In [ ]:
user_location_cols = [col for col in geocov_df.columns if col.endswith('user_location')]
place_cols = [col for col in geocov_df.columns if col.endswith('place')]
coordinates_cols = [col for col in geocov_df.columns if col.endswith('coordinates')]

for col in user_location_cols + place_cols + coordinates_cols:
#     geocov_df[col].apply(clean_list)
    geocov_df[col] = geocov_df[col].apply(clean_list)

In [ ]:
geocov_df['All User FIPS from user_location'] = geocov_df[user_location_cols].sum(axis=1)
geocov_df['All User FIPS from place'] = geocov_df[place_cols].sum(axis=1)
geocov_df['All User FIPS from coordinates'] = geocov_df[coordinates_cols].sum(axis=1)

geocov_df['ALL LOCATIONS'] = geocov_df[['All User FIPS from user_location', 'All User FIPS from place', 'All User FIPS from coordinates']].sum(axis=1)

In [ ]:
# geocov_df

In [ ]:
def do_locs_agree(location_list):
    return location_list.count(location_list[0]) == len(location_list)

In [ ]:
geocov_df['Consistent Location'] = geocov_df['ALL LOCATIONS'].apply(do_locs_agree)

In [ ]:
geocov_df = geocov_df.reindex(sorted(geocov_df.columns, reverse=True), axis=1)

In [ ]:
# geocov_df[geocov_df['All User FIPS from coordinates'].map(len) != 0]

# Merge with Master User List

In [ ]:
# MASTER_USER_LIST = pd.read_csv(r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Geo Cross Referencing/Master User List - GEOLOCATED.csv")
MASTER_USER_LIST = pd.read_csv(r"/arc/project/st-tlemieux-1/data/Master-User-Info.csv")

In [ ]:
MASTER_USER_LIST['Weighted CT Score'] = (MASTER_USER_LIST['Num CT Tweets - HT'] / MASTER_USER_LIST['Num CT Tweets - HT'].mean()).fillna(0) + (MASTER_USER_LIST['Num CT Tweets - LINK'] / MASTER_USER_LIST['Num CT Tweets - LINK'].mean()).fillna(0)

In [ ]:
MASTER_USER_LIST = MASTER_USER_LIST.merge(geocov_df, how='left', left_on='ID', right_index=True)

### See how many have locations

In [ ]:
# MASTER_USER_LIST[~MASTER_USER_LIST['ALL LOCATIONS'].isnull()]

In [ ]:
MASTER_USER_LIST.to_csv(r"/scratch/st-tlemieux-1/lfrymire/geocov_matching/Final-GeoCov-Output.csv", index=False)